In [401]:
import progressbar
import pickle
from cryptographic_estimators.SDEstimator import SDEstimator

In [402]:
def write_dict(d):
    with open('data_concrete.pkl', 'wb') as file:
        pickle.dump(d, file)
        
def read_dict():
    with open('data_concrete.pkl', 'rb') as file:
        loaded_dict = pickle.load(file)
    return loaded_dict

# Concrete Formulas of Different Algorithms

In [403]:
from math import log2, comb as binom, floor as fl, ceil as ce

def N(x):
    return x

def floor(x):
    return int(fl(x))

def ceil(x):
    return int(ce(x))

def r_int(x):
    return int(round(x))


def binomial(a,b):
    return binom(int(a),int(b))
#transform into regular instance and add new parity chekcs
def adapt_parameters(n,k,w):

    #Adjust parameters to consider non regularity and compute new code dimension
    r = n - k;
    b = floor(n/w);
    new_n = w*b;
    new_k = new_n-r;

    return new_n, new_k, b;

###############################################

#concrete cost estimate for permutation based ISD
def permutation_based_concrete_cost(n,k,w,b):


    k_prime  = k-w; #add parity-checks

    #success pr
    p_iter = (1-k_prime/n)**w;

    #cost of one iteration
    T_iter = (n-k_prime)**2*n;

    #overall cost
    cost = log2(T_iter)-log2(p_iter);

    return [cost, log2(n-k)+log2(n)];

###########################################################

#concrete cost estimate for enumeration based ISD
def enumeration_based_concrete_cost(n,k,w,b):


    k_prime = k-w;  #add parity-checks

    min_cost = [1000000000000000000000000000000,0];
    best_params = [];

    #	for p in range(0,floor(w/2)):

    p_max = 30;
    for p in range(0,p_max,2):

        #we first find a suitable range for the optimal value of ell
        L1_approx = max(1,binomial(r_int(w/2),p/2)*(k_prime/w)**(p/2));
        ell = log2(L1_approx);
        ell_min = ceil(ell*0.5);
        ell_max = min(r_int(ell*1.5), n-k_prime);


        for ell in range(ell_min, ell_max):


            v = (k_prime+ell)/w; #number of coordinates per block

            #success probability
            if w/2<p/2 or v/b>=1:
                continue
            p_iter = log2(binomial(floor(w/2), r_int(p/2)))+log2(binomial(ceil(w/2), r_int(p/2)))+log2(v/b)*p+log2(1-v/b)*(w-p);

            #cost of one iteration
            L = log2(binomial(r_int(w/2),p/2))+log2(v)*(p/2);
            T_iter = log2(n)+max(log2(n-k_prime)*2, 1+L,L*2-ell);

            #overall cost
            cost = T_iter-p_iter;

            #update min cost  if new cost is smaller
            if cost<min_cost[0]:
                min_cost = [N(cost),max(L,log2(n-k))+log2(n)]
                best_params = {"p":p, "ell": ell};

    if best_params["p"]== p_max:
        print("!!!!ERROR: p_max is too low for enum-based");
    return best_params, min_cost


###########################################################################################


###########################################################à

#concrete cost estimate for representation based ISD
def representation_based_concrete_cost(n,k,w,b):


    k_prime = k-w;  #add parity-checks

    min_cost = [1000000000000000000000000000000,0];
    best_params = [];

    #	for p in range(0,floor(w/2)):
    #the algorithm raises an error in case the chosen ranges are not valid
    p_max = 40;
    eps_x_max = 32;
    eps_y_max = 20;

    for p in range(0,p_max,8):
        for eps_x in range(0,eps_x_max,4):

            p_x = p/2 + eps_x; #weight

            for eps_y in range(0,eps_y_max):

                p_y = p_x/2 + eps_y;

                v_approx = k_prime/w;
                L1 = log2(max(binomial(r_int(w/2),p_y//2)*k_prime**(p_y//2),1));
                ell_approx = r_int( 2*L1);

                ell_min = ceil(ell_approx*0.5);

                ell_max = floor(ell_approx*1.5);
                for ell in range(ell_min, ell_max):

                    v = (k_prime+ell)/w;

                    #Num repr
                    
                    if p/2 < p/4 or w/2-p/2<eps_x/2 or p_x/2<p_x/2 or w/2-p_x/2< eps_y/2:
                        continue
                    R_x = (log2(binomial(p//2, p//4))+log2(binomial(w//2-p//2, eps_x/2))+log2(v)*(eps_x/2))*2;
                    R_y = (log2(binomial(p_x//2, p_x//4))+log2(binomial(w//2-p_x//2, eps_y//2))+log2(v)*(eps_y//2))*2;

                    ell_x = floor(R_x);
                    ell_y = floor(R_y);

                    if ell_y>ell_x:
                        continue
                        
                    if w/2<p/2 or v==0 or v>=b:
                        continue
                   
                    # success probability
                    p_iter = log2(binomial(floor(w/2), r_int(p/2)))+log2(binomial(ceil(w/2), r_int(p/2)))+log2(v/b)*p+log2(1-v/b)*(w-p);
                
                    L1 = log2(binomial(r_int(w/2),p_y/2))+log2(v)*(p_y/2); #list size, first level (initial lists)
                    if -p_iter+L1>min_cost[0]:
                        continue

                    L_y1 = L1*2-ell_y;

                    N_y = L_y1*2-(ell_x-ell_y);

                    L_x1 = log2(binomial(r_int(w/2), p_x//2))*2+log2(v)*p_x-ell_x;

                    N_x = L_x1*2-(ell-ell_x);
                    if v==b:
                        continue

                    # cost of one iteration
                    T_gauss=log2(n-k_prime)*2
                    T_iter = log2(n)+max(T_gauss,3+L1,2+L_y1,1+N_y,1+L_x1,N_x);

                    # overall cost
                    cost = T_iter-p_iter;

                    # update min cost  if new cost is smaller
                    if cost<min_cost[0]:
                        min_cost = [N(cost), max(L1,L_y1,L_x1)+log2(n)];
                        best_params = {"p":p, "eps_x":eps_x, "eps_y":eps_y, "ell_x":ell_x, "ell_y":ell_y, "ell_min":ell_min, "ell":ell, "ell_max":ell_max};

    if best_params["p"]== p_max:
        print("!!!!ERROR: p_max is too low for repr-based");

    if best_params["eps_x"]== eps_x_max:
        print("!!!!ERROR: eps_x_max is too low for repr-based");

    if best_params["eps_y"]== eps_y_max:
        print("!!!!ERROR: eps_y_max is too low for repr-based");

    return best_params, min_cost

#####################################################################

def representation_based_concrete_cost_depth_2(n,k,w,b):


    k_prime = k-w;  #add parity-checks

    min_cost = [1000000000000000000000000000000,0];
    best_params = [0,0,0,0,0,0];

    #	for p in range(0,floor(w/2),2):
    #the algorithm raises an error in case the chosen ranges are not valid
    p_max = 40;
    eps_x_max = 32;

    for p in range(0,p_max,4):
        for eps_x in range(0,eps_x_max,2):

            p_x = p/2 + eps_x; #weight

            v_approx = k_prime/w;
            L1_approx=binomial(round(w/2),p_x/2)*k_prime**(p_x/2)
            if L1_approx<=0:
                continue
            if w/2<p/2:
                continue
            R = (binomial(p/2,p/4)*binomial(w/2-p/2,eps_x/2))**2
            ell_approx = ceil( 2*log2(L1_approx));
            ell_min = floor(ell_approx*0.5);
            ell_max = floor(ell_approx*1.5);
            
            for ell in range(ell_min, ell_max):

                v = (k_prime+ell)/w;

                #Num repr
                if R<=0:
                    break
                ell_x = floor(log2(R))
                L1 = log2(binomial(round(w/2),p_x/2))+log2(v)*(p_x/2); #list size, first level (initial lists)

                L_x1 = L1*2-ell_x;

                N_x = L_x1*2-(ell-ell_x);

                #success probability
                if w/2<p/2 or v>=b:
                    continue
                p_iter = log2(binomial(floor(w/2), round(p/2)))+log2(binomial(ceil(w/2), round(p/2)))+log2((v/b))*p+log2(1-v/b)*(w-p);

                #cost of one iteration
                T_iter = log2(n)+ max(log2(n-k_prime)*2, 2+L1, 1+L_x1, N_x);

                #overall cost
                cost = T_iter-p_iter;

                #update min cost  if new cost is smaller
                if cost<min_cost[0]:
                    min_cost = [N(cost),max(L1,L_x1)+log2(n)];
                    best_params = {"p": p, "eps_x": eps_x, "ell_x": ell_x, "ell_min": ell_min, "ell": ell, "ell_max": ell_max};


    if best_params["p"]== p_max:
        print("!!!!ERROR: p_max is too low for repr-based");

    if best_params["eps_x"]== eps_x_max:
        print("!!!!ERROR: eps_x_max is too low for repr-based");

    # print(internal_values)
    return best_params, min_cost
#################################################################################

#concrete cost estimate for CCJ algorithm
def ccj_concrete_cost(n,k,w,b, nn_linear_time=0):


    min_cost = [1000000000000000000000000000000,0];
    best_params = [];

    #we first find a suitable range for the optimal value of ell
    k_tilde_approx = k-(1-k/n)/(1-w/n)*w;
    L1_approx = log2(n/w)*(w*k_tilde_approx/(2*n));
    ell_approx = L1_approx;
    ell_min = ceil(ell_approx*0.75);
    ell_max = min(r_int(ell_approx*1.5), n-k);

    if nn_linear_time:
        return [k_tilde_approx,0], L1_approx

    for ell in range(ell_min, ell_max):

        k_tilde = k-(1-(ell+k)/n)/(1-w/n)*w;
        if ell < (n-k_tilde):

            #cost of one iteration
            L = (n/w)**(w*(k_tilde+ell)/(2*n));
            num_coll = L**2*2**-ell;
            T_iter = (n-k_tilde)**2*n+n*(2*L+num_coll);

            #overall cost
            cost = log2(T_iter);

            #update min cost  if new cost is smaller
            if cost<min_cost[0]:
                min_cost = [cost,log2(L)+log2(n)];
                best_params = {"k_tilde": k_tilde, "ell": ell};

    return best_params, min_cost

#################################################################################

#concrete cost estimate for CCJ linearizatoin algorithm
def ccj_lin_concrete_cost(n,k,w,b):
    min_cost = [1000000000000000000000000000000,0];
    best_params = [];
    
    # r=n-k
    # n=n-w
    # k=n-r
    # b=b-1
    iterations = log2((n-w)/(n-k))*(w*(1-w/n))
    #cost of one iteration
    T_iter = (n-k)**2*n;

    #overall cost
    cost = log2(T_iter)+iterations;

    return [cost, log2(n-k)+log2(n)];

# List of RSD parameters from the literature

In [405]:
#Format for all lists is [n, k, w, claimed_security_level]
#Parameters from https://eprint.iacr.org/2022/712.pdf
hardness_lpn_larger_weight = [[2**10, 652, 106, 146], [2**12, 1589, 172, 131], [2**14, 3482, 338, 132], [2**16, 7391, 667, 135], [2**18, 15336, 1312, 123], [2**20, 32771, 2467, 126], [2**22, 64770, 4788, 103]];
hardness_lpn_smaller_weight = [[2**10, 652, 57, 94], [2**12, 1589, 98, 83], [2**14, 3482, 198, 86], [2**16, 7391, 389, 91], [2**18, 15336, 760, 95], [2**20, 32771, 1419, 98], [2**22, 64770, 2735, 103]];

# Parameters  from Tables 1 and 2 of the paper https://eprint.iacr.org/2018/208.pdf 
tiny_keys_GMW = [[245760, 245460, 15],
 [1006632960, 1006631460, 15],
 [40960, 40660, 20],
 [85899345920, 85899342920, 20],
 [7680, 7380, 30],
 [1966080, 1964080, 30],
 [5120, 4720, 40],
 [1310720, 1307720, 40],
 [3200, 2750, 50],
 [12800, 11800, 50],
 [1280, 860, 80],
 [20480, 17980, 80]];
tiny_keys_GMW = [i+[128] for i in tiny_keys_GMW]

tiny_keys_BMR = [[42949672960, 42949671678, 10],
 [5905580032, 5905578870, 11],
 [4194304, 4193582, 16],
 [42949672960, 42949669758, 10],
 [2684354560, 2684351858, 20],
 [1638400, 1636798, 25],
 [10240, 9438, 40],
 [68719476736, 68719471614, 16],
 [25769803776, 25769798974, 24],
 [4194304, 4191582, 32],
 [14336, 13054, 56],
 [85899345920, 85899339518, 20],
 [1006632960, 1006627958, 30],
 [1310720, 1307718, 40],
 [15360, 13758, 60],
 [983040, 977438, 60],
 [25600, 22398, 100],
 [163840, 155038, 80],
 [30720, 24318, 120],
 [11520, 6718, 180]]
tiny_keys_BMR = [i+[128] for i in tiny_keys_BMR]

# Parameters from paper https://eprint.iacr.org/2019/1159.pdf
BCGIKRS = [[20000, 10000, 73, 80],
 [200000, 100000, 72, 80],
 [2000000, 1000000, 70, 80],
 [20000000, 10000000, 68, 80],
 [40000, 30000, 37, 80],
 [400000, 300000, 36, 80],
 [4000000, 3000000, 35, 80],
 [40000000, 30000000, 34, 80],
 [20000, 10000, 126, 128],
 [200000, 100000, 120, 128],
 [2000000, 1000000, 118, 128],
 [20000000, 10000000, 116, 128],
 [40000, 30000, 80, 128],
 [400000, 300000, 72, 128],
 [4000000, 3000000, 63, 128],
 [40000000, 30000000, 54, 128]]

# Parameters from the paper https://eprint.iacr.org/2022/1014.pdf (aggressive parameters)
BCGIKRS22 = [[2*2**i[0],2**i[0],i[1],128] for i in [[20,75],[25,72],[30,68]]]+[[5*2**i[0],2**i[0],i[1],128] for i in [[20,94],[25,90],[30,85]]]

# Parameters from https://eprint.iacr.org/2023/1035.pdf
ccj = [[1842, 825, 307, 128]];

# Parameters from https://eprint.iacr.org/2020/924.pdf
ferret=[[609728, 36288,1269,128],[10805248, 589760, 1319,128]]

# Set up dicitionary

In [407]:
all_params = {
    "PCG_large_weight": hardness_lpn_larger_weight,
 "PCG_small_weight": hardness_lpn_smaller_weight, 
 "tiny_keys_GMW":tiny_keys_GMW,
 "tiny_keys_BMR":tiny_keys_BMR,
 "BCGIKRS":BCGIKRS,
 "BCGIKRS22":BCGIKRS22,
 "ccj":ccj,
 "ferret":ferret
}

In [408]:
all_estimates = {}

In [409]:
number_params=0
for i in all_params:
    c=1
    all_estimates[i]={}
    for j in all_params[i]:
        all_estimates[i][c]={}
        all_estimates[i][c]["params"]=j  
        all_estimates[i][c]["estimates"]={}
        all_estimates[i][c]["estimates"]["BJMM"]={}
        all_estimates[i][c]["estimates"]["CCJ"]={}
        all_estimates[i][c]["estimates"]["CCJ-lin"]={}
        all_estimates[i][c]["estimates"]["Perm"]={}
        all_estimates[i][c]["estimates"]["Enum"]={}
        all_estimates[i][c]["estimates"]["Rep"]={}
        all_estimates[i][c]["estimates"]["RepD2"]={}
        c+=1
        number_params+=1
            

# Start costs estimate

### You can just load the data from the provided dump if you want (instead of recomputing)
For loading execute next cell and skip the one after

In [420]:
all_estimates=read_dict()

In [410]:
rows = [];
c=1

bar = progressbar.ProgressBar(maxval=number_params)
bar.start()
for i in all_estimates:
    for j in all_estimates[i]:
        
        n,k,w,claimed_security=all_estimates[i][j]["params"]

        #Adapt parameters
        new_n, new_k, b = adapt_parameters(n,k,w);

      #  print(new_n, new_k, b)

        if new_k/new_n<1:

            #Computing BJMM using CryptographicEstimator
            A=SDEstimator(n,k,w,nsolutions = 0)
            bjmm_cost = A.bjmm.BJMM_depth_2
            
            if new_n > 10**4:
                bjmm_cost.set_parameters({"r":10})
            bjmm = bjmm_cost.time_complexity()
            # print("BJMM done")
            
            
            ccj_params, ccj_cost = ccj_concrete_cost(new_n, new_k, w, b);
            ccj_lin_cost = ccj_lin_concrete_cost(new_n, new_k, w, b);
            #Computing costs for regular ISD
            perm_cost = permutation_based_concrete_cost(new_n, new_k, w, b);
            enum_params, enum_cost = enumeration_based_concrete_cost(new_n, new_k, w, b);
            repr_params, repr_cost = representation_based_concrete_cost(new_n, new_k, w, b);
            repr_paramsd2, repr_costd2 = representation_based_concrete_cost_depth_2(new_n, new_k, w, b);

            
            #Consider DOOM if code is quasi-cyclic
            if i == "BCGIKRS":
                p = n-k; #circulant size
                ccj_cost[0] = ccj_cost[0] - 0.5*log2(p);
                bjmm = bjmm - 0.5*log2(p);
                perm_cost[0] = perm_cost[0] - 0.5*log2(p);
                enum_cost[0] = enum_cost[0] - 0.5*log2(p);
                repr_cost[0] = repr_cost[0] - 0.5*log2(p);
                repr_costd2[0] = repr_costd2[0] - 0.5*log2(p);

            all_estimates[i][j]["estimates"]["BJMM"]["time"]=int(r_int(bjmm))
            all_estimates[i][j]["estimates"]["BJMM"]["memory"]=int(r_int(bjmm_cost.memory_complexity()))
            all_estimates[i][j]["estimates"]["BJMM"]["params"]=bjmm_cost.optimal_parameters()

            all_estimates[i][j]["estimates"]["CCJ"]["time"]=r_int(ccj_cost[0])
            all_estimates[i][j]["estimates"]["CCJ"]["memory"]=r_int(ccj_cost[1])
            all_estimates[i][j]["estimates"]["CCJ"]["params"]=ccj_params      
            
            all_estimates[i][j]["estimates"]["CCJ-lin"]["time"]=r_int(ccj_lin_cost[0])
            all_estimates[i][j]["estimates"]["CCJ-lin"]["memory"]=r_int(ccj_lin_cost[1])
            
            all_estimates[i][j]["estimates"]["Perm"]["time"]=r_int(perm_cost[0])
            all_estimates[i][j]["estimates"]["Perm"]["memory"]=r_int(perm_cost[1])
            
            all_estimates[i][j]["estimates"]["Enum"]["time"]=r_int(enum_cost[0])
            all_estimates[i][j]["estimates"]["Enum"]["memory"]=r_int(enum_cost[1])
            all_estimates[i][j]["estimates"]["Enum"]["params"]=enum_params   
            all_estimates[i][j]["estimates"]["Rep"]["time"]=r_int(repr_cost[0])
            all_estimates[i][j]["estimates"]["Rep"]["memory"]=r_int(repr_cost[1])
            all_estimates[i][j]["estimates"]["Rep"]["params"]=repr_params   
            
            all_estimates[i][j]["estimates"]["RepD2"]["time"]=r_int(repr_costd2[0])
            all_estimates[i][j]["estimates"]["RepD2"]["memory"]=r_int(repr_costd2[1])
            all_estimates[i][j]["estimates"]["RepD2"]["params"]=repr_paramsd2   

            bar.update(c)
            c+=1



### Add algebraic attacks to dictionary

In [421]:
# costs of algebraic attacks from https://eprint.iacr.org/2023/176.pdf
algebraic_cost = [[145,101], [135,103], [138,106], [139,108], [122,104], [125,98], [103,103]];

names = ['BJMM','Algebraic', 'CCJ', 'CCJ-lin', 'Perm', 'Enum', 'Rep', 'RepD2']
for i in all_estimates["PCG_large_weight"]:
    all_estimates["PCG_large_weight"][i]["estimates"]["Algebraic"]={}
    all_estimates["PCG_large_weight"][i]["estimates"]["Algebraic"]["time"]=algebraic_cost[i-1][0]
    all_estimates["PCG_large_weight"][i]["estimates"]["Algebraic"]["memory"]="-"
    all_estimates["PCG_small_weight"][i]["estimates"]["Algebraic"]={}
    all_estimates["PCG_small_weight"][i]["estimates"]["Algebraic"]["time"]=algebraic_cost[i-1][1]
    all_estimates["PCG_small_weight"][i]["estimates"]["Algebraic"]["memory"]="-"
    all_estimates["PCG_large_weight"][i]["estimates"]={j: all_estimates["PCG_large_weight"][i]["estimates"][j] for j in names}
    all_estimates["PCG_small_weight"][i]["estimates"]={j: all_estimates["PCG_small_weight"][i]["estimates"][j] for j in names}


# Printing only time

In [422]:
from prettytable import PrettyTable

max_rate=1
algorithms = [[i for i in all_estimates[j][1]["estimates"] if i!= "RepD2"] for j in all_estimates]
row_header = ["params"]+ algorithms+["prev best","now best"]

c=0
for i in all_estimates:
    print("\n\n")
    print("---------------------------------")
    print(i) 
    print("---------------------------------")
    row_header = ["params"]+ algorithms[c]+["prev","now"]
    t = PrettyTable(row_header)
    for j in all_estimates[i]:
        par = [all_estimates[i][j]["params"]]
        if par[0][1]/par[0][0]>max_rate:
            continue
        times = [all_estimates[i][j]["estimates"][k]["time"] for k in algorithms[c]]
        if j in ["PCG_large_weight", "PCG_small_weight"]:
            t.add_row(par+times+[min(times[:4]),min(times)])
        else:
            t.add_row(par+times+[min(times[:3]),min(times)])


    print(t)
    c+=1




---------------------------------
PCG_large_weight
---------------------------------
+-----------------------------+------+-----------+-----+---------+------+------+-----+------+-----+
|            params           | BJMM | Algebraic | CCJ | CCJ-lin | Perm | Enum | Rep | prev | now |
+-----------------------------+------+-----------+-----+---------+------+------+-----+------+-----+
|    [1024, 652, 106, 146]    | 164  |    145    | 129 |   139   | 133  | 115  | 113 | 129  | 113 |
|    [4096, 1589, 172, 131]   | 135  |    135    | 160 |   132   | 131  | 110  | 109 | 135  | 109 |
|   [16384, 3482, 338, 132]   | 135  |    138    | 204 |   141   | 140  | 118  | 118 | 135  | 118 |
|   [65536, 7391, 667, 135]   | 139  |    139    | 249 |   149   | 149  | 126  | 127 | 139  | 126 |
|  [262144, 15336, 1312, 123] | 144  |    122    | 274 |   150   | 150  | 126  | 128 | 122  | 122 |
| [1048576, 32771, 2467, 126] | 148  |    125    | 335 |   164   | 164  | 138  | 141 | 125  | 125 |
| [4194304, 

# Printing time and memory

In [423]:
from prettytable import PrettyTable

max_rate=1
algorithms_index = [[i for i in all_estimates[j][1]["estimates"] if i!= "RepD2"] for j in all_estimates]
algorithms = [[[i + " time",i +" mem"] for i in all_estimates[j][1]["estimates"] if i!= "RepD2"] for j in all_estimates]
algorithms = [ [a for tup in j for a in tup] for j in algorithms]
row_header = ["params"]+ algorithms#+["prev","now"]

c=0
for i in all_estimates:
    print("\n\n")
    print("---------------------------------")
    print(i) 
    print("---------------------------------")
    row_header = ["params"]+ algorithms[c]#+["prev","now"]
    t = PrettyTable(row_header)
    for j in all_estimates[i]:
        par = [all_estimates[i][j]["params"]]
        if par[0][1]/par[0][0]>max_rate:
            continue
        times_mem = [[all_estimates[i][j]["estimates"][k]["time"],all_estimates[i][j]["estimates"][k]["memory"]] for k in algorithms_index[c]]
        times_mem_expanded = [a for tup in times_mem for a in tup]
        t.add_row(par+times_mem_expanded)
        # if j in ["PCG_large_weight", "PCG_small_weight"]:
        #     t.add_row(par+times+[min(times[:4]),min(times)])
        # else:
        #     t.add_row(par+times+[min(times[:3]),min(times)])


    print(t)
    c+=1




---------------------------------
PCG_large_weight
---------------------------------
+-----------------------------+-----------+----------+----------------+---------------+----------+---------+--------------+-------------+-----------+----------+-----------+----------+----------+---------+
|            params           | BJMM time | BJMM mem | Algebraic time | Algebraic mem | CCJ time | CCJ mem | CCJ-lin time | CCJ-lin mem | Perm time | Perm mem | Enum time | Enum mem | Rep time | Rep mem |
+-----------------------------+-----------+----------+----------------+---------------+----------+---------+--------------+-------------+-----------+----------+-----------+----------+----------+---------+
|    [1024, 652, 106, 146]    |    164    |   128    |      145       |       -       |   129    |   128   |     139      |      18     |    133    |    18    |    115    |    31    |   113    |    60   |
|    [4096, 1589, 172, 131]   |    135    |    51    |      135       |       -       |   16

# Print for use in latex tables

## Only time

In [424]:
algorithms = [[i for i in all_estimates[j][1]["estimates"] if i!= "RepD2"] for j in all_estimates]

print_choices = {"PCG_large_weight": [i for i in all_estimates["PCG_large_weight"]],
 "PCG_small_weight": [i for i in all_estimates["PCG_small_weight"]], 
 "tiny_keys_GMW":[1,3,5,11],
 "tiny_keys_BMR":[],
 "BCGIKRS":[],
 "BCGIKRS22":[],
 "ccj":[1],
 "ferret":[1,2]}

def print_n(n):
    if log2(n)%1==0:
        return "2^{"+str(int(log2(n)))+"}"
    else:
        return str(n)
    
def underline(x,v):
    if x == v:
        return "\\textbf{"+str(x)+"}"
    else:
        return x
c=0
for i in all_estimates:
    for j in all_estimates[i]:
        if j not in print_choices[i]:
                continue
        n,k,w, sec = all_estimates[i][j]["params"]
        times = [all_estimates[i][j]["estimates"][k]["time"] for k in algorithms[c]]

        if i in ["PCG_large_weight","PCG_small_weight"]:
            print('{0: <32}'.format("$("+print_n(n)+", "+str(k)+", "+str(w)+")$"), end="")
        else:
            print('{0: <32}'.format("& $("+print_n(n)+", "+str(k)+", "+str(w)+")$"), end="")
        mini = min(times)
        for o in times:
            print('{0: <17}'.format("&"+str(underline(o,mini))), end="")
        print("\\\\")
        
    if print_choices[i]!=[]:
        if i != "PCG_small_weight":
            print("\\midrule")
        else:
            print("\n\n\n")
    c+=1

$(2^{10}, 652, 106)$            &164             &145             &129             &139             &133             &115             &\textbf{113}    \\
$(2^{12}, 1589, 172)$           &135             &135             &160             &132             &131             &110             &\textbf{109}    \\
$(2^{14}, 3482, 338)$           &135             &138             &204             &141             &140             &\textbf{118}    &\textbf{118}    \\
$(2^{16}, 7391, 667)$           &139             &139             &249             &149             &149             &\textbf{126}    &127             \\
$(2^{18}, 15336, 1312)$         &144             &\textbf{122}    &274             &150             &150             &126             &128             \\
$(2^{20}, 32771, 2467)$         &148             &\textbf{125}    &335             &164             &164             &138             &141             \\
$(2^{22}, 64770, 4788)$         &149             &\textbf{103}    &360      

## Time and Memory

In [425]:
algorithms = [[i for i in all_estimates[j][1]["estimates"] if i!= "RepD2" and i!= "Algebraic"] for j in all_estimates]

print_choices = {"PCG_large_weight": [1],
 "PCG_small_weight": [1], 
 "tiny_keys_GMW":[1,3,5,11],
 "tiny_keys_BMR":[],
 "BCGIKRS":[],
 "BCGIKRS22":[],
 "ccj":[1],
 "ferret":[1,2]}

def print_n(n):
    if log2(n)%1==0:
        return "2^{"+str(int(log2(n)))+"}"
    else:
        return str(n)
    
def underline(x,v):
    if x == v:
        return "\\textbf{"+str(x)+"}"
    else:
        return x
c=0
for i in all_estimates:
    for j in all_estimates[i]:
        if j not in print_choices[i]:
                continue
        n,k,w, sec = all_estimates[i][j]["params"]
        times = [all_estimates[i][j]["estimates"][k]["time"] for k in algorithms[c]]
        memory = [all_estimates[i][j]["estimates"][k]["memory"] for k in algorithms[c]]

        print('{0: <32}'.format("$("+print_n(n)+", "+str(k)+", "+str(w)+")$"), end="")
        mini = min(times)
        count=0
        for o in times:
            print('{0: <17}'.format("&"+str(underline(o,mini))), end="")
            
            if count in [0,4,5]:
                try:
                    print('{0: <17}'.format("&"+str(memory[count])), end="")
                except:
                    print(c)
            count+=1
        print("\\\\")
        
    if print_choices[i]!=[]:
        if i != "PCG_small_weight":
            print("\\midrule")
        else:
            print("\n\n\n")
    c+=1

$(2^{10}, 652, 106)$            &164             &128             &129             &139             &133             &115             &31              &\textbf{113}    &60              \\
\midrule
$(2^{10}, 652, 57)$             &94              &51              &90              &96              &94              &77              &31              &\textbf{76}     &47              \\




$(245760, 245460, 15)$          &127             &99              &\textbf{124}    &180             &179             &127             &119             &140             &79              \\
$(40960, 40660, 20)$            &\textbf{126}    &100             &\textbf{126}    &174             &172             &\textbf{126}    &125             &129             &107             \\
$(7680, 7380, 30)$              &\textbf{127}    &117             &131             &169             &166             &132             &117             &132             &114             \\
$(1280, 860, 80)$               &134           

## Dump the current dictionary to file

In [418]:
write_dict(all_estimates)